In [18]:
from keras.preprocessing import sequence
from keras.models import Model, Input
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout, LSTM, Conv1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

In [22]:
rawdata = pd.read_csv("CUTe_data.csv")

In [23]:
rawdata.head()

,id,text,hate_speech,obscene,insulting
0,0,"""\n\nWikipedia an interesting yet frustrating ...",1,0,0
1,1,"""\n\nThanks\nThanks for reverting the vandalis...",0,0,0
2,2,(UTC)\n\nAnd fundamental Christains are not as...,1,1,1
3,3,"Why don't you go fuck your mom 05:52, 26 Jun ...",1,1,0
4,4,"Not to mention, he's a nobody. He's his bigges...",0,0,0


In [24]:
maxlen  = 300 # Max sequence length of a sentence or row/document.
max_features  = 20000 # This picks the top 20,000 most frequent words
embedding_dims  = 60
VALIDATION_SPLIT = 0.2

In [25]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(rawdata.text)
sequences = tokenizer.texts_to_sequences(rawdata.text)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

# labels = to_categorical(raw.Labels)
# print('Shape of data tensor:', data.shape)
# print('Shape of label tensor:', labels.shape)

Found 67252 unique tokens.


In [26]:
num_validation_samples = int(VALIDATION_SPLIT*data.shape[0])

In [27]:
x_train = data[:-num_validation_samples]

In [28]:
x_train.shape

(25308, 300)

In [29]:
y = rawdata[["hate_speech","obscene","insulting"]]

In [30]:
y.head()

,hate_speech,obscene,insulting
0,1,0,0
1,0,0,0
2,1,1,1
3,1,1,0
4,0,0,0


In [31]:
x_train= data[:-num_validation_samples]
x_test= data[-num_validation_samples:]
y_train= y[:-num_validation_samples]
y_test= y[-num_validation_samples:]

In [32]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(25308, 300)
(6326, 300)
(25308, 3)
(6326, 3)


In [15]:
sequence_input = Input((maxlen,))

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
X = Embedding(max_features, embedding_dims, input_length=maxlen)(sequence_input)
#                         embeddings_initializer="uniform")(comment_input)

X = Conv1D(64, 5, activation='relu')(X)

# we add a GlobalMaxPooling1D, which will extract features from the embeddings
# of all words in the comment
h = GlobalMaxPooling1D()(X)

comment_emb = Dropout(0.1)(h)
comment_emb= Dense(50, activation="relu")(comment_emb)
comment_emb = Dropout(0.1)(comment_emb)

# We project onto a six-unit output layer, and squash it with a sigmoid:
output = Dense(3, activation='sigmoid')(h)

model = Model(inputs=sequence_input, outputs=output)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.01),
              metrics=['accuracy'])

In [16]:
    model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 300, 200)          4000000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 296, 64)           64064     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 195       
Total params: 4,064,259
Trainable params: 4,064,259
Non-trainable params: 0
_________________________________________________________________


In [17]:
model_hist = model.fit(x_train, y_train,
              batch_size=32,
              epochs=2,
              validation_data=(x_test, y_test)).history

Train on 25308 samples, validate on 6326 samples
Epoch 1/2
25308/25308 [==============================] - 202s 8ms/step - loss: 0.3151 - acc: 0.8685 - val_loss: 0.2979 - val_acc: 0.8762
Epoch 2/2
25308/25308 [==============================] - 211s 8ms/step - loss: 0.2312 - acc: 0.9089 - val_loss: 0.3163 - val_acc: 0.8778


In [41]:
from keras.layers import LeakyReLU, BatchNormalization

sequence_input = Input((maxlen,))

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
X = Embedding(max_features, embedding_dims, input_length=maxlen)(sequence_input)
#                         embeddings_initializer="uniform")(comment_input)

X = Conv1D(64, 5,  activation='linear')(X)
X = LeakyReLU(alpha=0.1)(X)
# we add a GlobalMaxPooling1D, which will extract features from the embeddings
# of all words in the comment
h = GlobalMaxPooling1D()(X)

comment_emb = Dropout(0.1)(h)
#comment_emb = BatchNormalization()(comment_emb)

comment_emb= Dense(50, activation='linear')(comment_emb)
comment_emb = LeakyReLU(alpha=0.1)(comment_emb)
comment_emb = Dropout(0.1)(comment_emb)
#comment_emb = BatchNormalization()(comment_emb)

# We project onto a six-unit output layer, and squash it with a sigmoid:
output = Dense(3, activation='sigmoid')(h)

model = Model(inputs=sequence_input, outputs=output)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.01),
              metrics=['accuracy'])

In [35]:
#droput 0.1
model_hist = model.fit(x_train, y_train,
              batch_size=32,
              epochs=10,
              validation_data=(x_test, y_test)).history

Train on 25308 samples, validate on 6326 samples
Epoch 1/10
25308/25308 [==============================] - 95s 4ms/step - loss: 0.1650 - acc: 0.9364 - val_loss: 0.3668 - val_acc: 0.8737
Epoch 2/10
25308/25308 [==============================] - 98s 4ms/step - loss: 0.1280 - acc: 0.9541 - val_loss: 0.4087 - val_acc: 0.8762
Epoch 3/10
25308/25308 [==============================] - 94s 4ms/step - loss: 0.1010 - acc: 0.9664 - val_loss: 0.4824 - val_acc: 0.8744
Epoch 4/10
25308/25308 [==============================] - 84s 3ms/step - loss: 0.0867 - acc: 0.9726 - val_loss: 0.5391 - val_acc: 0.8734
Epoch 5/10
25308/25308 [==============================] - 85s 3ms/step - loss: 0.0832 - acc: 0.9753 - val_loss: 0.5911 - val_acc: 0.8745
Epoch 6/10
25308/25308 [==============================] - 94s 4ms/step - loss: 0.0752 - acc: 0.9784 - val_loss: 0.7038 - val_acc: 0.8768
Epoch 7/10
25308/25308 [==============================] - 96s 4ms/step - loss: 0.0761 - acc: 0.9791 - val_loss: 0.7665 - val_acc: